In [1]:
%matplotlib inline

import quandl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from talib import RSI, BBANDS, MACD
import datetime as dt

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [12]:
max_holding = 100

source = "CSV"

symbol = 'NSE/INFY'

In [3]:
if source == "CSV":
    df = pd.read_csv("nse_50.csv")
    df = df.set_index('Date')

    #Convert date from string to date format if required - uncomment below line
    df.index = [dt.datetime.strptime(date, '%Y-%m-%d') for date in df.index]

    price = pd.DataFrame({'open':df["Open"],'high':df["High"],'low':df["Low"],'close':df["Close"],'adjClose':df["Adj Close"],"volume":df["Volume"]})


In [4]:
#Fetch Data and pre-process data - Source:Quandl: NSE
if source == "Quandl":
    start = '2018-03-24'
    end = '2019-03-24'

    symbol = 'NSE/INFY'
    max_holding = 100
    df = web.DataReader(name=symbol, data_source='quandl', start=start, end=end,  access_key="B2LcBEywuebxqE9s15L9")

    #Convert date from string to date format if required - uncomment below line
    #df.index = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in df.index]

    price = pd.DataFrame({'open':df["Open"],'high':df["High"],'low':df["Low"],'close':df["Close"],'adjClose':df["Close"],"volume":df["TotalTradeQuantity"]})


In [5]:
price.head()

,open,high,low,close,adjClose,volume
2014-03-21,6515.200195,6522.899902,6485.700195,6493.200195,6493.200195,189900.0
2014-03-22,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-24,6510.500000,6591.500000,6510.500000,6583.500000,6583.500000,158400.0
2014-03-25,6550.100098,6595.549805,6544.850098,6589.750000,6589.750000,168100.0
2014-03-26,6615.649902,6627.450195,6580.600098,6601.399902,6601.399902,186600.0


In [6]:
#Clean up and filter data
price = price[price.index> dt.datetime(2018,1,1)]
price = price.iloc[::-1]
price = price.dropna()
close = price["adjClose"].values

In [7]:
macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

In [8]:
up, mid, low = BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
rsi = RSI(close, timeperiod=14)
print("RSI (first 10 elements)\n", rsi[14:24])

def bbp(price):
    up, mid, low = BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    bbp = (price["adjClose"] - low) / (up - low)
    return bbp
price['RSI']=rsi
price['BBP']=bbp(price)

holdings = pd.DataFrame(index=price.index, data={'Holdings': np.array([np.nan] * price.index.shape[0])})

RSI (first 10 elements)
 [ 5.95569949  9.40234032 14.66877364 13.0552102  13.0237992  12.07838015
 10.16356142 14.25068457 22.86713984 24.97342215]


In [9]:
holdings.loc[((price['RSI'] < 30) & (price['BBP'] < 0)), 'Holdings'] = max_holding
holdings.loc[((price['RSI'] > 70) & (price['BBP'] > 1)), 'Holdings'] = 0
holdings.ffill(inplace=True)
holdings.fillna(0, inplace=True)

holdings['Order'] = holdings.diff()
holdings.dropna(inplace=True)

In [10]:
data1 = []
data2 = []
for day, holding in holdings.iterrows():
    order = holding['Order']
    #print(day,order, price.loc[day, 'Adj Close'])
    if order > 0:
        data1.append((day, price.loc[day, 'adjClose']))
    elif order < 0:
        data2.append((day, price.loc[day, 'adjClose']))
        
data1 = np.asarray(data1)
data2 = np.asarray(data2)

In [14]:
trace0 = go.Ohlc(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Closing Price")

#trace0 = go.Scatter(x=price.index, y=price['Adj Close'], mode='lines', line=dict(color='rgb(63, 72, 204)'),name='Closing Price',yaxis="y1")

trace61 = go.Scatter(x=data1[::-1,0], y=data1[::-1,1], marker = dict(color='green', size=10), mode='markers')
#trace62 = go.Scatter(x=data2[::-1,0], y=data2[::-1,1], marker = dict(color='red', size=10), mode='markers')

trace1 = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI')

trace2 = go.Scatter(x=price.index, y=macd, name='MACD')
trace3 = go.Bar(x=price.index, y=macdhist, name='MACD Hist')
trace4 = go.Scatter(x=price.index, y=macdsignal, name='MACD signal')


price['BB_up'] = up
price['BB_low'] = low

trace5 = go.Scatter(x=price.index, y=price['BB_up'], name='BB_up')
#trace6 = go.Scatter(x=price.index, y=price['Adj Close'], name='AdjClose')
trace7 = go.Scatter(x=price.index, y=price['BB_low'], name='BB_low', fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)")

fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, shared_yaxes=False, vertical_spacing=0.05, row_width=[1,2,3])


fig.append_trace(trace0, 1, 1)
fig.append_trace(trace5, 1, 1)
fig.append_trace(trace7, 1, 1)
fig.append_trace(trace61, 1, 1)
#fig.append_trace(trace62, 1, 1)

fig.append_trace(trace1, 3, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 1)

fig['layout'].update(height=750, title="Stock Analysis for "+symbol)
fig['layout']['xaxis']['rangeslider'].update(thickness=0.05)
#fig['layout']['yaxis1'].update(range=[8000,12000], domain=[0.5,1.0])
#fig['layout']['yaxis'].update(domain=[0.10,0.15])
#fig['layout']['yaxis2'].update(domain=[0.9,1])
#fig['layout'].update(annotations=[dict(text="Buy",x=data1[::-1,0],y=data1[::-1,1])])

iplot(fig, filename="Stock Analysis 2")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]



In [13]:
help(tools.make_subplots)

Help on function make_subplots in module plotly.tools:

make_subplots(rows=1, cols=1, shared_xaxes=False, shared_yaxes=False, start_cell='top-left', print_grid=True, **kwargs)
    Return an instance of plotly.graph_objs.Figure
    with the subplots domain set in 'layout'.
    
    Example 1:
    # stack two subplots vertically
    fig = tools.make_subplots(rows=2)
    
    This is the format of your plot grid:
    [ (1,1) x1,y1 ]
    [ (2,1) x2,y2 ]
    
    fig['data'] += [Scatter(x=[1,2,3], y=[2,1,2])]
    fig['data'] += [Scatter(x=[1,2,3], y=[2,1,2], xaxis='x2', yaxis='y2')]
    
    # or see Figure.append_trace
    
    Example 2:
    # subplots with shared x axes
    fig = tools.make_subplots(rows=2, shared_xaxes=True)
    
    This is the format of your plot grid:
    [ (1,1) x1,y1 ]
    [ (2,1) x1,y2 ]
    
    
    fig['data'] += [Scatter(x=[1,2,3], y=[2,1,2])]
    fig['data'] += [Scatter(x=[1,2,3], y=[2,1,2], yaxis='y2')]
    
    Example 3:
    # irregular subplot layout (mor

In [45]:
import datetime as dt
print(price.loc["2018-9-3"]["Close"]
,price.loc["2018-9-4"]["Close"]
,price.loc["2018-9-5"]["Close"]
,price.loc["2018-9-6"]["Close"])

print(price.loc["2018-9-3"]["Last"]
,price.loc["2018-9-4"]["Last"]
,price.loc["2018-9-5"]["Last"]
,price.loc["2018-9-6"]["Last"])

1434.25 737.15 729.9 727.15
1436.2 733.0 729.0 726.0
